In [1]:
import pandas as pd

In [3]:
file_path='../Resources/yelpdata.csv'
yelp_df=pd.read_csv(file_path)
yelp_df.head(10)

,Unnamed: 0,latitude,longitude,category,results_total,avg_rating,avg_review_count
0,0,38.046407,-84.497039,beaches,1,1.500000,0.666667
1,1,38.046407,-84.497039,hanggliding,0,0.000000,0.000000
2,2,38.046407,-84.497039,horsebackriding,8,3.000000,0.666667
3,3,38.046407,-84.497039,hiking,4,6.166667,18.666667
4,4,38.046407,-84.497039,hot_air_baloons,20,28.333333,1723.000000
5,5,38.046407,-84.497039,paddleboarding,2,3.166667,10.000000
6,6,38.046407,-84.497039,parasailing,0,0.000000,0.000000
7,7,38.046407,-84.497039,sailing,0,0.000000,0.000000
8,8,38.046407,-84.497039,snorkeling,0,0.000000,0.000000
9,9,38.046407,-84.497039,ziplining,20,28.333333,1723.000000


In [4]:
yelp_df = yelp_df.drop(yelp_df.columns[0], axis=1)
yelp_df.head()

,latitude,longitude,category,results_total,avg_rating,avg_review_count
0,38.046407,-84.497039,beaches,1,1.500000,0.666667
1,38.046407,-84.497039,hanggliding,0,0.000000,0.000000
2,38.046407,-84.497039,horsebackriding,8,3.000000,0.666667
3,38.046407,-84.497039,hiking,4,6.166667,18.666667
4,38.046407,-84.497039,hot_air_baloons,20,28.333333,1723.000000


In [5]:
yelp_df['top_rated'] = yelp_df['avg_rating'] * yelp_df['avg_review_count'] 

# Groupby the geocoordinates and keep only the rows with the highest 5 'top_rated' values
yelp_df= yelp_df.groupby(['latitude', 'longitude']).apply(lambda x: x.nlargest(5, 'top_rated')).reset_index(drop=True)

yelp_df.head(10)

,latitude,longitude,category,results_total,avg_rating,avg_review_count,top_rated
0,-45.032192,168.661000,hot_air_baloons,20,29.333333,504.666667,14803.555556
1,-45.032192,168.661000,ziplining,20,29.333333,504.666667,14803.555556
2,-45.032192,168.661000,transportation,20,29.333333,504.666667,14803.555556
3,-45.032192,168.661000,restaurants,20,29.166667,507.000000,14787.500000
4,-45.032192,168.661000,nightlife,20,28.333333,194.666667,5515.555556
5,-37.814218,144.963161,hot_air_baloons,20,28.666667,747.666667,21433.111111
6,-37.814218,144.963161,ziplining,20,28.666667,747.666667,21433.111111
7,-37.814218,144.963161,transportation,20,28.666667,747.666667,21433.111111
8,-37.814218,144.963161,restaurants,20,28.666667,741.000000,21242.000000
9,-37.814218,144.963161,nightlife,20,28.166667,416.666667,11736.111111


In [6]:
# Group by 'latitude' and 'longitude' columns and aggregate other columns
aggregated_df = yelp_df.groupby(['latitude', 'longitude']).agg({'category': list,
                                                          'results_total': 'sum',
                                                          'avg_rating': 'mean',
                                                          'avg_review_count': 'mean',
                                                          'top_rated': lambda x: x.any()}).reset_index()

# Get the maximum number of categories
max_categories = max(aggregated_df['category'].apply(len))

# Loop through the categories and create separate columns
for i in range(max_categories):
    aggregated_df['category{}'.format(i + 1)] = aggregated_df['category'].apply(lambda x: x[i] if len(x) > i else '')

# Drop the original 'category' column
aggregated_df = aggregated_df.drop('category', axis=1)

aggregated_df.head()


,latitude,longitude,results_total,avg_rating,avg_review_count,top_rated,category1,category2,category3,category4,category5
0,-45.032192,168.661000,100,29.100000,443.133333,True,hot_air_baloons,ziplining,transportation,restaurants,nightlife
1,-37.814218,144.963161,100,28.566667,680.133333,True,hot_air_baloons,ziplining,transportation,restaurants,nightlife
2,-36.852095,174.763180,100,28.966667,433.733333,True,hot_air_baloons,ziplining,transportation,restaurants,nightlife
3,-36.718805,142.220951,100,24.166667,12.466667,True,hot_air_baloons,ziplining,transportation,restaurants,hotels
4,-35.141813,150.391646,81,5.800000,3.800000,True,restaurants,nightlife,hotels,resorts,beaches
